# LAB: Utilizando Pandas con datos de oferta de departamentos en la CABA

## Introducción

El objetivo de este LAB es trabajar con los conceptos de agregación a través de `groupby` y `pivot tables` mencionadas en la sección teórica. 

> Para eso usaremos un dataset elaborado por el Gobierno de la Ciudad de Buenos Aires que releva el precio de oferta de venta de terrenos de la Ciudad Autónoma de Buenos Aires, representado en puntos de valor del suelo correspondientes a cada parcela. Con el fin de analizar el precio de oferta, se toma como unidad de referencia el precio del metro cuadrado (m2). Se utiliza el precio en dólares, puesto que ésta es la moneda que se usa en el mercado como referencia para este tipo de transacciones. Otras variables consideradas son las dimensiones, ubicación geográfica, y la normativa del Código de Planeamiento Urbano que rige la zona en que el terreno analizado se localiza. 

Pueden descargar el dataset y otros elaborados para los años 2001-2015 desde este [link](
https://data.buenosaires.gob.ar/dataset/departamentos-en-venta). En el pdf adjunto pueden encontrar una descripción sucinta de las variables relevadas.

Importamos los paquetes necesarios y cargamos los datos.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

In [3]:
df = pd.read_csv("ED_deptos_venta_2014.csv",encoding = "latin1", sep=";")

In [4]:
# Veamos los tipos de las columnas...
df.dtypes

CALLE          object
NUMERO          int64
M2              int64
DOLARES         int64
U_S_M2        float64
AMBIENTES       int64
ANTIGUEDAD    float64
ORIENT         object
BAULERA        object
COCHERA        object
BAÃ.OS          int64
LAVADERO       object
TERRAZA        object
BARRIO         object
COMUNA          int64
LON           float64
LAT           float64
dtype: object

In [5]:
# Revisamos que los datos se han importado correctamente
df.head()

,CALLE,NUMERO,M2,DOLARES,U_S_M2,AMBIENTES,ANTIGUEDAD,ORIENT,BAULERA,COCHERA,BAÃ.OS,LAVADERO,TERRAZA,BARRIO,COMUNA,LON,LAT
1,11 DE SEPTIEMBRE DE 1888,700,560,1270000,2267.9,5,20.0,FRENTE,Si,Si,4,No,Si,PALERMO,14,-58.439523,-34.570692
2,11 DE SEPTIEMBRE DE 1888,800,126,295000,2341.3,4,45.0,S/D,No,Si,0,No,No,PALERMO,14,-58.440256,-34.570313
3,11 DE SEPTIEMBRE DE 1888,900,109,320000,2935.8,4,20.0,FRENTE,Si,Si,3,Si,Si,PALERMO,14,-58.441321,-34.569487
4,11 DE SEPTIEMBRE DE 1888,900,150,400000,2666.7,4,37.0,FRENTE,Si,Si,2,Si,Si,PALERMO,14,-58.441321,-34.569487
5,11 DE SEPTIEMBRE DE 1888,900,270,950000,3518.5,4,30.0,FRENTE,No,No,3,No,Si,PALERMO,14,-58.441321,-34.569487


### 1. ¿Cuál es el valor promedio del metro cuadrado en la CABA?

In [6]:
df["U_S_M2"].mean()

2320.5095253065069

### 2. ¿Cómo es el precio promedio para cada cuartil de antigüedad de las viviendas en la CABA?

In [8]:
q_ant = pd.qcut(df['ANTIGUEDAD'], 4)
df.groupby(q_ant)['DOLARES'].mean()

ANTIGUEDAD
(-0.001, 5.0]    183477.107300
(5.0, 30.0]      250570.962068
(30.0, 40.0]     180525.217157
(40.0, 115.0]    212822.615945
Name: DOLARES, dtype: float64

### 3. ¿Cómo es el precio promedio para cada decil de tamaño (columna M2)?

In [10]:
q_m = pd.qcut(df['M2'], 10)
df.groupby(q_m)['DOLARES'].mean()

M2
(13.999, 34.0]      72342.990719
(34.0, 40.0]        85343.722772
(40.0, 47.0]        99520.104380
(47.0, 55.0]       112046.517564
(55.0, 63.0]       130499.336339
(63.0, 74.0]       152753.276744
(74.0, 89.1]       173960.781775
(89.1, 115.0]      226988.144186
(115.0, 170.0]     343407.225730
(170.0, 1500.0]    717688.368254
Name: DOLARES, dtype: float64

### 4. ¿Cuál es la media de precio por metro cuadrado para cada barrio de la CABA? Ordenar los datos para indicar cuál es el barrio más caro.
Nota: realizar el cálculo tanto con groupby como con pivot tables

In [14]:
df.groupby('BARRIO')['U_S_M2'].mean().sort_values(ascending = False)

BARRIO
PUERTO MADERO        5582.909615
PALERMO              2944.154531
RETIRO               2811.813858
RECOLETA             2748.067676
NUÃEZ               2670.226065
BELGRANO             2611.026478
SAAVEDRA             2455.130702
COLEGIALES           2421.881746
VILLA DEVOTO         2417.066327
VILLA URQUIZA        2343.938785
COGHLAN              2331.871429
VILLA ORTUZAR        2331.174265
CABALLITO            2285.865212
PARQUE CHAS          2269.022535
VILLA PUEYRREDON     2222.173469
VILLA CRESPO         2185.012876
CHACARITA            2184.093939
PARQUE CHACABUCO     2105.960769
ALMAGRO              2077.175568
SAN NICOLAS          2070.846863
AGRONOMIA            2057.252381
SAN TELMO            2042.093651
BARRACAS             2012.427797
BOEDO                2006.131579
VILLA DEL PARQUE     2004.857252
VILLA LURO           1980.403636
VILLA GRAL. MITRE    1926.297638
MONTE CASTRO         1924.870192
FLORES               1912.556360
SAN CRISTOBAL        1865.804500
LIN

In [21]:
df.pivot_table(index = 'BARRIO', aggfunc={'U_S_M2':'mean'}).sort_values(by='U_S_M2',ascending=False)

,U_S_M2
BARRIO,
PUERTO MADERO,5582.909615
PALERMO,2944.154531
RETIRO,2811.813858
RECOLETA,2748.067676
NUÃEZ,2670.226065
BELGRANO,2611.026478
SAAVEDRA,2455.130702
COLEGIALES,2421.881746
VILLA DEVOTO,2417.066327


### 5. ¿En qué barrio hay una mayor dispersión de precios de metro cuadrado? Ordenar los valores para identificar el mayor.

(Recordar la fórmula del coeficiente de variación para medir la dispersión)

**Pistas**
* Una primera opción es definir una función con arrays y utilizar `.apply()`

In [26]:
df.groupby('BARRIO')['U_S_M2'].apply(lambda x: x.std() / x.mean()).sort_values(ascending = False)

BARRIO
VILLA SOLDATI        0.405491
RETIRO               0.343333
VILLA LUGANO         0.325593
CONSTITUCION         0.314560
BARRACAS             0.314212
PALERMO              0.307419
VILLA DEVOTO         0.307213
MONTSERRAT           0.300801
BOCA                 0.296929
SAAVEDRA             0.295136
BOEDO                0.295016
SAN TELMO            0.290003
SAN CRISTOBAL        0.277870
NUEVA POMPEYA        0.276016
BELGRANO             0.273090
VILLA LURO           0.272364
VILLA RIACHUELO      0.270715
VILLA CRESPO         0.269843
RECOLETA             0.263554
VILLA DEL PARQUE     0.260707
VILLA PUEYRREDON     0.259385
BALVANERA            0.256931
MATADEROS            0.255745
LINIERS              0.249943
PARQUE PATRICIOS     0.247107
PUERTO MADERO        0.243482
PARQUE CHACABUCO     0.240950
VILLA GRAL. MITRE    0.239854
NUÃEZ               0.236443
VERSALLES            0.234789
VILLA ORTUZAR        0.232977
CHACARITA            0.231876
SAN NICOLAS          0.225197
ALM

* Una segunda, es generar dos series: 
  - una con el método `.std()` y dividirla por otra serie generada con `mean()`

**Bonus:** ¿cuál le parece mejor y por qué?

### 6. ¿En qué barrio los departamentos son más grandes? 
Nota: se puede realizar el cálculo tanto con groupby como con pivot tables

In [27]:
df.pivot_table('M2', index='BARRIO', aggfunc=np.median).sort_values(by='M2', ascending=False)

,M2
BARRIO,
RETIRO,139.0
PUERTO MADERO,136.5
RECOLETA,100.0
PALERMO,95.0
BARRACAS,75.0
BELGRANO,75.0
VILLA DEVOTO,72.0
SAN TELMO,70.0
AGRONOMIA,65.0


In [28]:
df.groupby('BARRIO')['M2'].median().sort_values(ascending=False)

BARRIO
RETIRO               139.0
PUERTO MADERO        136.5
RECOLETA             100.0
PALERMO               95.0
BARRACAS              75.0
BELGRANO              75.0
VILLA DEVOTO          72.0
SAN TELMO             70.0
MONTSERRAT            65.0
VILLA DEL PARQUE      65.0
AGRONOMIA             65.0
MATADEROS             64.0
COLEGIALES            63.5
VERSALLES             63.0
PATERNAL              63.0
CABALLITO             62.0
BOCA                  61.0
LINIERS               60.0
VILLA CRESPO          60.0
PARQUE PATRICIOS      60.0
FLORES                60.0
VILLA LURO            60.0
BALVANERA             60.0
PARQUE CHACABUCO      58.0
COGHLAN               58.0
CONSTITUCION          57.0
NUÃEZ                56.0
MONTE CASTRO          56.0
PARQUE AVELLANEDA     56.0
SAAVEDRA              55.5
ALMAGRO               55.0
VILLA SOLDATI         55.0
VILLA PUEYRREDON      55.0
VELEZ SARSFIELD       55.0
VILLA GRAL. MITRE     55.0
VILLA ORTUZAR         54.5
VILLA LUGANO         

### 7. En general, puede observarse alguna diferencia entre el precio mediano de aquellos departamentos que tienen terraza y los que no? ¿Y en el tamaño?

In [29]:
df.groupby('TERRAZA')['U_S_M2'].median()

TERRAZA
No    2141.25
Si    2264.80
Name: U_S_M2, dtype: float64

In [30]:
df.groupby('TERRAZA')['M2'].median()

TERRAZA
No    60
Si    66
Name: M2, dtype: int64

### 8. Genere un `DataFrame` que agregue la información del precio por M2 (`U_S_M2`), ambientes (`AMBIENTES`) y antigüedad (`ANTIGUEDAD`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

In [31]:
df.pivot_table(['U_S_M2','AMBIENTES','ANTIGUEDAD'],index=['COMUNA','BARRIO']
               , aggfunc={'U_S_M2':[np.mean,np.std,len],
                          'AMBIENTES': [np.mean,np.std],
                          'ANTIGUEDAD': [np.mean,np.std]})

AMBIENTES           ANTIGUEDAD             U_S_M2  \
                              mean       std       mean        std     len   
COMUNA BARRIO                                                                
1      CONSTITUCION       0.861878  1.432843  46.849673  28.399817   181.0   
       MONTSERRAT         1.474104  1.937608  54.950820  27.679738   251.0   
       PUERTO MADERO      2.173077  1.748466   4.494253   3.399294   104.0   
       RETIRO             2.756554  2.049368  47.022831  21.263408   267.0   
       SAN NICOLAS        0.523985  1.352082  49.139130  19.034965   271.0   
       SAN TELMO          2.793651  2.080313  41.254902  31.537814    63.0   
2      RECOLETA           3.116757  1.723089  39.678763  21.241869   925.0   
3      BALVANERA          2.359452  1.451531  38.904580  22.352470  1021.0   
       SAN CRISTOBAL      1.125000  1.385269  26.550000  22.710040   200.0   
4      BARRACAS           2.783051  1.461659  30.668317  23.712286   295.0   
       BOCA               2.492147  1.383623  33.719298  20.697787   191.0   
       NUEVA POMPEYA      1.105263  1.448008  27.931034  15.622628    38.0   
       PARQUE PATRICIOS   0.861111  1.397236  27.032258  22.836531    72.0   
5      ALMAGRO            2.434595  1.228844  28.647139  20.432820   925.0   
       BOEDO              2.110048  1.169461  23.964539  20.334141   209.0   
6      CABALLITO          1.867779  1.825055  23.334795  20.323386   779.0   
7      FLORES             0.233216  0.863432  23.663498  19.051781   566.0   
       PARQUE CHACABUCO   0.915385  1.436159  19.525000  20.066680   130.0   
8      VILLA LUGANO       2.625000  0.957427  24.100000  12.982467    16.0   
       VILLA RIACHUELO    2.600000  0.547723  28.000000  20.720360     5.0   
       VILLA SOLDATI      3.500000  0.849837  31.500000  10.239978    10.0   
9      LINIERS            2.571429  1.291330  13.565217  17.437701    56.0   
       MATADEROS          2.489583  0.973272  18.166667  17.768692    96.0   
       PARQUE AVELLANEDA  0.824561  1.269460  25.977778  21.443890    57.0   
10     FLORESTA           0.188034  0.819289  26.490741  16.182080   117.0   
       MONTE CASTRO       1.403846  1.376073  14.271429  16.393637   104.0   
       VELEZ SARSFIELD    0.558442  1.219291  28.855072  19.254698    77.0   
       VERSALLES          1.960000  1.513275  20.565217  19.453502    25.0   
       VILLA LURO         2.345455  1.350147  17.360000  18.874191    55.0   
       VILLA REAL         1.888889  1.364225  23.375000  19.003289     9.0   
11     VILLA DEL PARQUE   2.442748  0.985747  18.555556  19.473804   131.0   
       VILLA DEVOTO       3.091837  1.324525  19.302326  16.258985    98.0   
       VILLA GRAL. MITRE  1.039370  1.595637  24.663636  17.604356   127.0   
       VILLA SANTA RITA   1.163636  1.243705  19.304348  18.861919    55.0   
12     COGHLAN            1.076923  1.484967  12.697368  14.405573    91.0   
       SAAVEDRA           1.228070  1.634988  12.504950  15.579232   114.0   
       VILLA PUEYRREDON   2.346939  1.109636  11.179487  14.256802    49.0   
       VILLA URQUIZA      2.252336  1.099654  19.843305  16.613625   642.0   
13     BELGRANO           2.855296  1.368827  25.339639  18.303213  1624.0   
       COLEGIALES         2.087302  1.605791  25.922830  17.670291   378.0   
       NUÃEZ             0.924812  1.481667  16.097859  15.646681   399.0   
14     PALERMO            2.954299  1.720894  27.056202  18.678835  1291.0   
15     AGRONOMIA          2.523810  1.030488  15.250000  18.035151    21.0   
       CHACARITA          0.439394  1.140895  19.081301  20.050180   132.0   
       PARQUE CHAS        2.070423  1.099662  19.756757  16.135546    71.0   
       PATERNAL           0.352941  0.996317  24.533333  20.749756    17.0   
       VILLA CRESPO       2.193133  1.687056  24.504950  20.585521   233.0   
       VILLA ORTUZAR      2.213235  1.416349  18.386364  17.816067   136.0   

                                              